# Texas Crash Prediction - Instructor Setup & Pipeline Notebook

**Team Road Watch | SIADS 699 Capstone**

---

## Overview

This notebook provides an interactive, end-to-end walkthrough for:

1. **Environment Setup** - Verify Python environment and dependencies
2. **Data Directory Setup** - Create medallion architecture directories
3. **Data Verification** - Validate required data files are in place
4. **Data Pipeline** - Build ML-ready crash-level datasets
5. **Model Training** - Train and evaluate crash severity prediction models
6. **Streamlit App** - Launch the interactive dashboard

## Prerequisites

- **Python 3.10+**
- **~5GB disk space** for data and models
- **Data files** downloaded per `DATA_ACQUISITION.md`

## Time Estimates

| Mode | Pipeline | Training | Total |
|------|----------|----------|-------|
| Sample (10K) | ~2 min | ~5 min | ~10 min |
| Full (466K) | ~30 min | ~15 min | ~1 hour |

---

## Configuration

Adjust these settings before running the notebook:

In [2]:
# =============================================================================
# CONFIGURATION - Edit these settings
# =============================================================================

# Sample mode: Set to True for quick demos (~10 min), False for full run (~1 hour)
SAMPLE_MODE = True

# Number of samples to use in sample mode
SAMPLE_SIZE = 10_000

# Which models to train: 'baseline', 'xgboost', 'catboost', 'lightgbm', 'all'
MODELS_TO_TRAIN = 'baseline'  # 'baseline' trains LogisticRegression + RandomForest

# Skip segment-level dataset (faster, crash-level is sufficient for demo)
SKIP_SEGMENT_LEVEL = True

# Skip training if you just want to run the pipeline
SKIP_TRAINING = False

---

## Section 1: Environment Setup

In [3]:
# Cell 1.1: Add project root to Python path
import sys
from pathlib import Path

# Ensure we're in the project directory
notebook_dir = Path.cwd()
if (notebook_dir / 'config' / 'paths.py').exists():
    PROJECT_ROOT = notebook_dir
elif (notebook_dir.parent / 'config' / 'paths.py').exists():
    PROJECT_ROOT = notebook_dir.parent
else:
    raise RuntimeError("Cannot find project root. Please run from project directory.")

# Add to Python path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")

Project root: /Users/julien.hovan/Desktop/MADS/Capstone/Capstone-SIADS699-Team-RoadWatch


In [4]:
# Cell 1.2: Check Python version
import platform

python_version = platform.python_version()
major, minor = map(int, python_version.split('.')[:2])

if major >= 3 and minor >= 10:
    print(f"Python {python_version}")
else:
    print(f"Python {python_version}")
    print("   Python 3.10+ is recommended")

Python 3.14.0


In [5]:
# Cell 1.3: Install package in editable mode (if not already installed)
import subprocess
import shutil

# Check if package is installed
try:
    from config.paths import PROJECT_ROOT as _
    print("Package already installed")
except ImportError:
    print("Installing package...")
    # Use uv if available, otherwise pip
    if shutil.which("uv"):
        !uv pip install -e .
    else:
        !pip install -e .
    print("Package installed")

Package already installed


In [6]:
# Cell 1.4: Verify core imports
import warnings
warnings.filterwarnings('ignore')

checks = {}

try:
    import pandas as pd
    checks['pandas'] = f"v{pd.__version__}"
except ImportError as e:
    checks['pandas'] = f"MISSING: {e}"

try:
    import numpy as np
    checks['numpy'] = f"v{np.__version__}"
except ImportError as e:
    checks['numpy'] = f"MISSING: {e}"

try:
    import sklearn
    checks['scikit-learn'] = f"v{sklearn.__version__}"
except ImportError as e:
    checks['scikit-learn'] = f"MISSING: {e}"

try:
    import geopandas as gpd
    checks['geopandas'] = f"v{gpd.__version__}"
except ImportError as e:
    checks['geopandas'] = f"MISSING: {e}"

try:
    import xgboost
    checks['xgboost'] = f"v{xgboost.__version__}"
except ImportError as e:
    checks['xgboost'] = f"MISSING: {e}"

try:
    import mlflow
    checks['mlflow'] = f"v{mlflow.__version__}"
except ImportError as e:
    checks['mlflow'] = f"MISSING: {e}"

try:
    import streamlit
    checks['streamlit'] = f"v{streamlit.__version__}"
except ImportError as e:
    checks['streamlit'] = f"MISSING: {e}"

print("Dependency Check:")
print("-" * 40)
all_ok = True
for pkg, status in checks.items():
    if "MISSING" in status:
        print(f"  {pkg}: {status}")
        all_ok = False
    else:
        print(f"  {pkg}: {status}")

if all_ok:
    print("\nAll dependencies available")
else:
    print("\nSome dependencies are missing. Run: pip install -e .")

Dependency Check:
----------------------------------------
  pandas: MISSING: No module named 'pandas'
  numpy: MISSING: No module named 'numpy'
  scikit-learn: MISSING: No module named 'sklearn'
  geopandas: MISSING: No module named 'geopandas'
  xgboost: MISSING: No module named 'xgboost'
  mlflow: MISSING: No module named 'mlflow'
  streamlit: MISSING: No module named 'streamlit'

Some dependencies are missing. Run: pip install -e .


In [ ]:
# Cell 1.5: macOS XGBoost check (libomp dependency)
import platform

if platform.system() == 'Darwin':
    print("macOS detected - checking XGBoost libomp dependency...")
    try:
        import xgboost
        # Try a simple operation to verify libomp
        import numpy as np
        xgb_test = xgboost.DMatrix(np.array([[1, 2], [3, 4]]))
        print("  XGBoost works correctly")
    except Exception as e:
        print(f"  XGBoost issue: {e}")
        print("  If XGBoost fails, try: brew install libomp")
else:
    print(f"Platform: {platform.system()} - XGBoost should work without additional setup")

---

## Section 2: Data Directory Setup

In [ ]:
# Cell 2.1: Create medallion architecture directories
from config.paths import (
    ensure_directories,
    DATA_ROOT,
    BRONZE, BRONZE_TEXAS,
    SILVER, SILVER_TEXAS,
    GOLD, GOLD_ML_DATASETS,
    CRASH_LEVEL_ML, SEGMENT_LEVEL_ML,
    TEXAS_BRONZE_CRASHES, TEXAS_SILVER_ROADWAY,
    DEFAULT_CRASH_FILE, DEFAULT_HPMS_FILE
)

print("Creating directory structure...\n")
ensure_directories()
print("Directory structure ready")

In [ ]:
# Cell 2.2: Display directory tree
def print_tree(directory, prefix='', max_depth=3, current_depth=0):
    """Print directory tree with depth limit"""
    if current_depth >= max_depth:
        return
    
    try:
        entries = sorted([e for e in directory.iterdir() if e.is_dir()])
    except PermissionError:
        return
    
    for i, entry in enumerate(entries):
        is_last = (i == len(entries) - 1)
        connector = '' if is_last else ''
        print(f"{prefix}{connector}{entry.name}/")
        
        extension = '    ' if is_last else '   '
        print_tree(entry, prefix + extension, max_depth, current_depth + 1)

print("Data Directory Structure:")
print(f"{DATA_ROOT.name}/")
print_tree(DATA_ROOT)

In [ ]:
# Cell 2.3: Show expected data file locations
print("Expected Data File Locations:")
print("=" * 60)
print()
print("Bronze Layer (raw data):")
print(f"  Crash data: {DEFAULT_CRASH_FILE}")
print()
print("Silver Layer (cleaned data):")
print(f"  HPMS data:  {DEFAULT_HPMS_FILE}")
print()
print("See DATA_ACQUISITION.md for download instructions.")

---

## Section 3: Data Verification

Verify that the required data files are in place before running the pipeline.

In [ ]:
# Cell 3.1: Check if data files exist
import pandas as pd

def check_file(file_path, description):
    """Check if file exists and return status"""
    if file_path.exists():
        size_mb = file_path.stat().st_size / (1024 * 1024)
        return True, f"{size_mb:.1f} MB"
    return False, "NOT FOUND"

print("Data File Verification:")
print("=" * 60)
print()

# Check crash data
crash_exists, crash_info = check_file(DEFAULT_CRASH_FILE, "Crash Data")
status = "" if crash_exists else ""
print(f"{status} Crash Data (Kaggle US Accidents): {crash_info}")
print(f"    Path: {DEFAULT_CRASH_FILE}")

print()

# Check HPMS data
hpms_exists, hpms_info = check_file(DEFAULT_HPMS_FILE, "HPMS Data")
status = "" if hpms_exists else ""
print(f"{status} HPMS Data (Texas 2023): {hpms_info}")
print(f"    Path: {DEFAULT_HPMS_FILE}")

print()

# Summary
if crash_exists and hpms_exists:
    print("All required data files found!")
    DATA_READY = True
else:
    print("Missing data files. See DATA_ACQUISITION.md for download instructions.")
    DATA_READY = False

In [ ]:
# Cell 3.2: Validate crash data structure
if DEFAULT_CRASH_FILE.exists():
    print("Validating crash data...")
    
    # Read sample to check structure
    df_sample = pd.read_csv(DEFAULT_CRASH_FILE, nrows=100)
    
    required_cols = ['ID', 'Start_Time', 'Start_Lat', 'Start_Lng', 'Severity']
    missing_cols = [col for col in required_cols if col not in df_sample.columns]
    
    if missing_cols:
        print(f"  Missing required columns: {missing_cols}")
    else:
        print(f"  Required columns present")
    
    # Count total rows
    print("  Counting total rows (this may take a moment)...")
    total_rows = sum(1 for _ in open(DEFAULT_CRASH_FILE)) - 1
    print(f"  Total crashes: {total_rows:,}")
    
    # Show sample
    print("\nSample data (first 3 rows):")
    display(df_sample[required_cols].head(3))
else:
    print("Crash data file not found. Skipping validation.")

In [ ]:
# Cell 3.3: Validate HPMS data structure
if DEFAULT_HPMS_FILE.exists():
    import geopandas as gpd
    
    print("Validating HPMS data...")
    
    # Read sample
    gdf_sample = gpd.read_file(DEFAULT_HPMS_FILE, rows=100)
    
    print(f"  Columns: {len(gdf_sample.columns)}")
    print(f"  CRS: {gdf_sample.crs}")
    print(f"  Geometry type: {gdf_sample.geometry.geom_type.mode()[0]}")
    
    # Count total features
    print("  Counting total road segments (this may take a moment)...")
    gdf_full = gpd.read_file(DEFAULT_HPMS_FILE)
    print(f"  Total road segments: {len(gdf_full):,}")
    
    # Key columns
    key_cols = ['speed_limit', 'through_lanes', 'aadt', 'f_system']
    available_key_cols = [col for col in key_cols if col in gdf_sample.columns]
    print(f"  Key columns available: {available_key_cols}")
else:
    print("HPMS data file not found. Skipping validation.")

---

## Section 4: Run Data Pipeline

Build ML-ready datasets from raw data. This section:
1. Loads crash data from Bronze layer
2. Integrates HPMS road characteristics
3. Engineers features
4. Creates train/val/test splits
5. Saves to Gold layer

In [1]:
# Cell 4.1: Pre-flight check before pipeline
print("Pipeline Pre-flight Check:")
print("=" * 60)
print()
print(f"Mode: {'SAMPLE' if SAMPLE_MODE else 'FULL'}")
if SAMPLE_MODE:
    print(f"Sample size: {SAMPLE_SIZE:,} records")
print(f"Skip segment-level: {SKIP_SEGMENT_LEVEL}")
print()

# Check if data is ready
if not (DEFAULT_CRASH_FILE.exists() and DEFAULT_HPMS_FILE.exists()):
    print("Data files not found! Cannot run pipeline.")
    print("Please download data per DATA_ACQUISITION.md")
    PIPELINE_READY = False
else:
    print("Data files found. Pipeline ready to run.")
    PIPELINE_READY = True

Pipeline Pre-flight Check:



NameError: name 'SAMPLE_MODE' is not defined

In [ ]:
# Cell 4.2: Run crash-level pipeline
import subprocess
import sys
import os

if PIPELINE_READY:
    print("Building Crash-Level Dataset")
    print("=" * 60)
    print()
    
    # Build command
    cmd = [sys.executable, 'data_engineering/datasets/build_crash_level_dataset.py']
    
    if SAMPLE_MODE:
        cmd.extend(['--sample', str(SAMPLE_SIZE)])
        print(f"Running in SAMPLE mode with {SAMPLE_SIZE:,} records...")
    else:
        print("Running in FULL mode (this will take ~30 minutes)...")
    
    print(f"\nCommand: {' '.join(cmd)}\n")
    print("-" * 60)
    
    # Run with live output
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        cwd=str(PROJECT_ROOT),
        env={**os.environ, 'PYTHONPATH': str(PROJECT_ROOT)}
    )
    
    # Stream output
    for line in process.stdout:
        print(line, end='')
    
    process.wait()
    
    print("-" * 60)
    if process.returncode == 0:
        print("\nCrash-level dataset built successfully!")
        CRASH_DATASET_BUILT = True
    else:
        print(f"\nPipeline failed with exit code {process.returncode}")
        CRASH_DATASET_BUILT = False
else:
    print("Pipeline not ready. Please check data files.")
    CRASH_DATASET_BUILT = False

In [ ]:
# Cell 4.3: Run segment-level pipeline (optional)
if PIPELINE_READY and not SKIP_SEGMENT_LEVEL:
    print("Building Segment-Level Dataset")
    print("=" * 60)
    print()
    
    cmd = [sys.executable, 'data_engineering/datasets/build_segment_level_dataset.py']
    
    if SAMPLE_MODE:
        cmd.extend(['--sample', str(SAMPLE_SIZE)])
    
    print(f"Command: {' '.join(cmd)}\n")
    print("-" * 60)
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        cwd=str(PROJECT_ROOT),
        env={**os.environ, 'PYTHONPATH': str(PROJECT_ROOT)}
    )
    
    for line in process.stdout:
        print(line, end='')
    
    process.wait()
    
    print("-" * 60)
    if process.returncode == 0:
        print("\nSegment-level dataset built successfully!")
    else:
        print(f"\nPipeline failed with exit code {process.returncode}")
else:
    print("Skipping segment-level dataset (SKIP_SEGMENT_LEVEL=True)")

---

## Section 5: Explore Generated Datasets

In [ ]:
# Cell 5.1: Check generated datasets
print("Generated Datasets:")
print("=" * 60)
print()

# Check crash-level datasets
crash_files = {
    'train': CRASH_LEVEL_ML / 'train_latest.csv',
    'val': CRASH_LEVEL_ML / 'val_latest.csv',
    'test': CRASH_LEVEL_ML / 'test_latest.csv'
}

print("Crash-Level Datasets:")
for split, path in crash_files.items():
    if path.exists():
        size_mb = path.stat().st_size / (1024 * 1024)
        df = pd.read_csv(path, nrows=1)
        total_rows = sum(1 for _ in open(path)) - 1
        print(f"  {split}: {total_rows:,} rows, {len(df.columns)} columns, {size_mb:.1f} MB")
    else:
        print(f"  {split}: NOT FOUND")

# Check segment-level datasets
if not SKIP_SEGMENT_LEVEL:
    print("\nSegment-Level Datasets:")
    segment_files = {
        'train': SEGMENT_LEVEL_ML / 'train_latest.csv',
        'val': SEGMENT_LEVEL_ML / 'val_latest.csv', 
        'test': SEGMENT_LEVEL_ML / 'test_latest.csv'
    }
    for split, path in segment_files.items():
        if path.exists():
            size_mb = path.stat().st_size / (1024 * 1024)
            print(f"  {split}: {size_mb:.1f} MB")
        else:
            print(f"  {split}: NOT FOUND")

In [ ]:
# Cell 5.2: Load and explore crash-level training data
train_path = CRASH_LEVEL_ML / 'train_latest.csv'

if train_path.exists():
    print("Loading training dataset...")
    train_df = pd.read_csv(train_path, low_memory=False)
    
    print(f"\nDataset shape: {train_df.shape}")
    print(f"Memory usage: {train_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    print("\nColumn Types:")
    print(train_df.dtypes.value_counts())
    
    print("\nFirst 5 rows:")
    display(train_df.head())
else:
    print("Training data not found. Run the pipeline first.")

In [ ]:
# Cell 5.3: Target variable distribution
if train_path.exists() and 'high_severity' in train_df.columns:
    import matplotlib.pyplot as plt
    
    print("Target Variable Distribution:")
    print("=" * 40)
    
    target_counts = train_df['high_severity'].value_counts()
    target_pct = train_df['high_severity'].value_counts(normalize=True) * 100
    
    print(f"\nClass 0 (Low Severity):  {target_counts[0]:,} ({target_pct[0]:.1f}%)")
    print(f"Class 1 (High Severity): {target_counts[1]:,} ({target_pct[1]:.1f}%)")
    
    # Simple bar chart
    fig, ax = plt.subplots(figsize=(8, 4))
    bars = ax.bar(['Low Severity (0)', 'High Severity (1)'], 
                  [target_counts[0], target_counts[1]],
                  color=['#2ecc71', '#e74c3c'])
    ax.set_ylabel('Count')
    ax.set_title('Target Variable Distribution')
    
    # Add count labels
    for bar, count in zip(bars, [target_counts[0], target_counts[1]]):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000,
                f'{count:,}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
else:
    print("Target variable not available.")

---

## Section 6: Model Training

Train crash severity prediction models with MLflow tracking.

In [ ]:
# Cell 6.1: Check if training should run
if SKIP_TRAINING:
    print("Training skipped (SKIP_TRAINING=True)")
    TRAINING_READY = False
elif not (CRASH_LEVEL_ML / 'train_latest.csv').exists():
    print("Training data not found. Run the pipeline first.")
    TRAINING_READY = False
else:
    print("Training Configuration:")
    print("=" * 40)
    print(f"Models to train: {MODELS_TO_TRAIN}")
    print(f"Dataset: crash-level")
    print()
    print("Ready to train!")
    TRAINING_READY = True

In [ ]:
# Cell 6.2: Train models
if TRAINING_READY:
    print("Training Models")
    print("=" * 60)
    print()
    
    cmd = [
        sys.executable, '-m', 'ml_engineering.train_with_mlflow',
        '--dataset', 'crash',
        '--model', MODELS_TO_TRAIN
    ]
    
    print(f"Command: {' '.join(cmd)}\n")
    print("-" * 60)
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        cwd=str(PROJECT_ROOT),
        env={**os.environ, 'PYTHONPATH': str(PROJECT_ROOT)}
    )
    
    for line in process.stdout:
        print(line, end='')
    
    process.wait()
    
    print("-" * 60)
    if process.returncode == 0:
        print("\nTraining completed successfully!")
        TRAINING_COMPLETE = True
    else:
        print(f"\nTraining failed with exit code {process.returncode}")
        TRAINING_COMPLETE = False
else:
    print("Training not ready or skipped.")
    TRAINING_COMPLETE = False

In [ ]:
# Cell 6.3: List trained models
from pathlib import Path

models_dir = PROJECT_ROOT / 'models' / 'artifacts'

if models_dir.exists():
    print("Trained Model Artifacts:")
    print("=" * 60)
    print()
    
    model_dirs = sorted([d for d in models_dir.iterdir() if d.is_dir()])
    
    for model_dir in model_dirs:
        model_file = model_dir / 'model.pkl'
        metrics_file = model_dir / 'metrics.json'
        
        if model_file.exists():
            size_mb = model_file.stat().st_size / (1024 * 1024)
            
            # Try to load metrics
            metrics_str = ""
            if metrics_file.exists():
                import json
                with open(metrics_file) as f:
                    metrics = json.load(f)
                    if 'roc_auc' in metrics:
                        metrics_str = f"AUC={metrics['roc_auc']:.3f}"
            
            print(f"  {model_dir.name}")
            print(f"      Size: {size_mb:.1f} MB  {metrics_str}")
else:
    print("No trained models found. Run training first.")

---

## Section 7: Launch Streamlit App

Start the interactive dashboard for exploring crash data and predictions.

In [ ]:
# Cell 7.1: Check Streamlit app
app_path = PROJECT_ROOT / 'app' / 'app.py'

if app_path.exists():
    print("Streamlit App Ready")
    print("=" * 40)
    print(f"\nApp location: {app_path}")
    print("\nTo launch manually, run:")
    print(f"  streamlit run {app_path}")
    STREAMLIT_READY = True
else:
    print(f"Streamlit app not found at {app_path}")
    STREAMLIT_READY = False

In [ ]:
# Cell 7.2: Launch Streamlit (runs in background)
import subprocess
import time

if STREAMLIT_READY:
    print("Launching Streamlit App...")
    print()
    
    # Start Streamlit in background
    streamlit_proc = subprocess.Popen(
        ['streamlit', 'run', str(app_path), '--server.headless=true'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        cwd=str(PROJECT_ROOT)
    )
    
    # Wait for server to start
    print("Waiting for server to start...")
    time.sleep(3)
    
    if streamlit_proc.poll() is None:  # Process still running
        print("\nStreamlit server started!")
        print()
        print("   Open in browser: http://localhost:8501")
        print()
        print("Run the next cell to stop the server when done.")
        
        # Store process reference
        _STREAMLIT_PROC = streamlit_proc
    else:
        print("Failed to start Streamlit. Check for errors.")
        stdout, stderr = streamlit_proc.communicate()
        print(stderr.decode())
else:
    print("Streamlit not ready.")

In [ ]:
# Cell 7.3: Stop Streamlit server
if '_STREAMLIT_PROC' in dir() and _STREAMLIT_PROC is not None:
    _STREAMLIT_PROC.terminate()
    _STREAMLIT_PROC.wait()
    print("Streamlit server stopped.")
    _STREAMLIT_PROC = None
else:
    print("No Streamlit server running.")

---

## Section 8: Validation Checklist

Summary of completed steps.

In [ ]:
# Cell 8.1: Final validation checklist
print("Validation Checklist")
print("=" * 60)
print()

# Check each item
checks = [
    ("Environment installed", True),  # If we got this far, environment is set up
    ("Data directories created", DATA_ROOT.exists()),
    ("Crash data available", DEFAULT_CRASH_FILE.exists()),
    ("HPMS data available", DEFAULT_HPMS_FILE.exists()),
    ("Pipeline completed", (CRASH_LEVEL_ML / 'train_latest.csv').exists()),
    ("Models trained", (PROJECT_ROOT / 'models' / 'artifacts').exists() and 
                       any((PROJECT_ROOT / 'models' / 'artifacts').iterdir())),
    ("Streamlit app available", (PROJECT_ROOT / 'app' / 'app.py').exists()),
]

for item, status in checks:
    icon = "" if status else ""
    print(f"  {icon} {item}")

print()
completed = sum(1 for _, s in checks if s)
total = len(checks)
print(f"Progress: {completed}/{total} steps complete")

if completed == total:
    print("\nAll steps completed! The project is fully set up.")

---

## Appendix: Useful Commands

Quick reference for common operations:

```bash
# Run full pipeline
python scripts/run_pipeline.py

# Run sample pipeline (faster)
python scripts/run_pipeline.py --sample 10000

# Train all models
python -m ml_engineering.train_with_mlflow --dataset crash --model all

# Train specific model
python -m ml_engineering.train_with_mlflow --dataset crash --model xgboost

# Launch Streamlit
streamlit run app/app.py

# View MLflow runs
mlflow ui
```

---

*Team Road Watch | SIADS 699 Capstone*